<a href="https://colab.research.google.com/github/eunha330/movieanalysis/blob/main/03_02_%EC%96%B8%EC%96%B4%EB%B6%84%EC%84%9D_%ED%95%9C%EA%B5%AD%EC%96%B4_%EC%8B%AC%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [2]:
# konlpy(한국어형태소분석기)를 실행하기
import konlpy

ModuleNotFoundError: ignored

In [3]:
# konlpy 설치하기 ## https://data1000.tistory.com/33
!pip3 install jpype1==0.7.0
!pip3 install konlpy

     |████████████████████████████████| 2.7MB 5.7MB/s 
     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 9.1MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [4]:
# konlpy를 실행하기
import konlpy

In [5]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [6]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [7]:
## 문서 ID로 실제 파일 불러오기
# 실습을 위한 역대 대통령 취임사: 노무현, 이명박, 박근혜, 문재인
## https://drive.google.com/file/d/19OXmMvT4gj9KA-Z5f2xLYk4TJkY_hLy6/view
rawdata_downloaded = drive.CreateFile({'id': '19OXmMvT4gj9KA-Z5f2xLYk4TJkY_hLy6'})
rawdata_downloaded.GetContentFile('rawdata.txt')

In [8]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('rawdata.txt', sep="\t")

In [9]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,"type,id,nickname,text,score,like,dislike,time,watch_movie"
0,"로망,0,한작가(hana****),""이런 영화 돈 안되는 거 알면서도 만든 제작자에..."
1,"로망,1,보리차(taij****),남들 눈엔 노망이라도. 나에게는 로망이기를.,8,..."
2,"로망,2,hkjj****,상영관이 생각보다 넘 없네요..볼만한 영화인거 같은데,9,..."
3,"로망,3,블리블리(stop****),억지 감동이 아닌 영화라 너무 좋았고 가족끼리 ..."
4,"로망,4,hsh1****,치매가 얼마나 무서운 질병인지 알려주고 부모님의 소중함과 ..."
...,...
21541,"감쪽같은그녀,1795,가는거야(lyjk****),대일본제국의 우수한 선진문화를 배워..."
21542,"감쪽같은그녀,1796,밍밍(ayeo****),ㅜㅜㅜ 완전 감동 폭발,10,4,10,..."
21543,"감쪽같은그녀,1797,fly1****,감동스럽고재밌고 그런영화예요마음따뜻해지는영화라..."
21544,"감쪽같은그녀,1798,멋진인생(dear****),감동도 재미도 아무것도 없다,1,0..."


# 형태소 분석

In [10]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [12]:
# "원본데이터"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터.iterrows(): # 원본데이터의 컬럼(열)을 불러와서 반복할 준비!
    대상텍스트 = row[0] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    분류 = row[0] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [13]:
# "형태소" 변수의 저장 내용 확인
형태소

[('감쪽', 'Noun'),
 ('같은', 'Adjective'),
 ('그녀', 'Noun'),
 (',', 'Punctuation'),
 ('1799', 'Number'),
 (',', 'Punctuation'),
 ('tang', 'Alpha'),
 ('****,,', 'Punctuation'),
 ('8', 'Number'),
 (',', 'Punctuation'),
 ('0', 'Number'),
 (',', 'Punctuation'),
 ('7,201', 'Number'),
 ('9-12', 'Number'),
 ('-', 'Punctuation'),
 ('09', 'Number'),
 ('03:06', 'Number'),
 (':', 'Punctuation'),
 ('00', 'Number'),
 (',', 'Punctuation'),
 ('관람객', 'Noun')]

In [14]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,감쪽,Noun,"감쪽같은그녀,1799,tang****,,8,0,7,2019-12-09 03:06:0...",1
1,같은,Adjective,"감쪽같은그녀,1799,tang****,,8,0,7,2019-12-09 03:06:0...",1
2,그녀,Noun,"감쪽같은그녀,1799,tang****,,8,0,7,2019-12-09 03:06:0...",1
3,",",Punctuation,"감쪽같은그녀,1799,tang****,,8,0,7,2019-12-09 03:06:0...",1
4,1799,Number,"감쪽같은그녀,1799,tang****,,8,0,7,2019-12-09 03:06:0...",1
5,",",Punctuation,"감쪽같은그녀,1799,tang****,,8,0,7,2019-12-09 03:06:0...",1
6,tang,Alpha,"감쪽같은그녀,1799,tang****,,8,0,7,2019-12-09 03:06:0...",1
7,"****,,",Punctuation,"감쪽같은그녀,1799,tang****,,8,0,7,2019-12-09 03:06:0...",1
8,8,Number,"감쪽같은그녀,1799,tang****,,8,0,7,2019-12-09 03:06:0...",1
9,",",Punctuation,"감쪽같은그녀,1799,tang****,,8,0,7,2019-12-09 03:06:0...",1


In [15]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[          0  ... count
 0        로망  ...     1
 1         ,  ...     1
 2         0  ...     1
 3         ,  ...     1
 4         한  ...     1
 5        작가  ...     1
 6         (  ...     1
 7      hana  ...     1
 8   ****),"  ...     1
 9        이런  ...     1
 10       영화  ...     1
 11        돈  ...     1
 12      안되는  ...     1
 13        거  ...     1
 14     알면서도  ...     1
 15       만든  ...     1
 16      제작자  ...     1
 17        에  ...     1
 18       박수  ...     1
 19        를  ...     1
 20    ,,,",  ...     1
 21    9,409  ...     1
 22        ,  ...     1
 23   13,201  ...     1
 24     9-04  ...     1
 25        -  ...     1
 26       03  ...     1
 27    16:00  ...     1
 28        :  ...     1
 29       00  ...     1
 30        ,  ...     1
 31        0  ...     1
 
 [32 rows x 4 columns],          0  ... count
 0       로망  ...     1
 1        ,  ...     1
 2        1  ...     1
 3        ,  ...     1
 4      보리차  ...     1
 5        (  ...     1
 6     taij  ...     1

In [16]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [17]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,로망,Noun,"로망,0,한작가(hana****),""이런 영화 돈 안되는 거 알면서도 만든 제작자에...",1
1,",",Punctuation,"로망,0,한작가(hana****),""이런 영화 돈 안되는 거 알면서도 만든 제작자에...",1
2,0,Number,"로망,0,한작가(hana****),""이런 영화 돈 안되는 거 알면서도 만든 제작자에...",1
3,",",Punctuation,"로망,0,한작가(hana****),""이런 영화 돈 안되는 거 알면서도 만든 제작자에...",1
4,한,Determiner,"로망,0,한작가(hana****),""이런 영화 돈 안되는 거 알면서도 만든 제작자에...",1
...,...,...,...,...
16,03:06,Number,"감쪽같은그녀,1799,tang****,,8,0,7,2019-12-09 03:06:0...",1
17,:,Punctuation,"감쪽같은그녀,1799,tang****,,8,0,7,2019-12-09 03:06:0...",1
18,00,Number,"감쪽같은그녀,1799,tang****,,8,0,7,2019-12-09 03:06:0...",1
19,",",Punctuation,"감쪽같은그녀,1799,tang****,,8,0,7,2019-12-09 03:06:0...",1


In [18]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [19]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,로망,Noun,"로망,0,한작가(hana****),""이런 영화 돈 안되는 거 알면서도 만든 제작자에...",1
1,",",Punctuation,"로망,0,한작가(hana****),""이런 영화 돈 안되는 거 알면서도 만든 제작자에...",1
2,0,Number,"로망,0,한작가(hana****),""이런 영화 돈 안되는 거 알면서도 만든 제작자에...",1
3,",",Punctuation,"로망,0,한작가(hana****),""이런 영화 돈 안되는 거 알면서도 만든 제작자에...",1
4,한,Determiner,"로망,0,한작가(hana****),""이런 영화 돈 안되는 거 알면서도 만든 제작자에...",1
...,...,...,...,...
16,03:06,Number,"감쪽같은그녀,1799,tang****,,8,0,7,2019-12-09 03:06:0...",1
17,:,Punctuation,"감쪽같은그녀,1799,tang****,,8,0,7,2019-12-09 03:06:0...",1
18,00,Number,"감쪽같은그녀,1799,tang****,,8,0,7,2019-12-09 03:06:0...",1
19,",",Punctuation,"감쪽같은그녀,1799,tang****,,8,0,7,2019-12-09 03:06:0...",1


In [20]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [21]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소  품사           분류                                                                                                                                                                                
!    Punctuation  감쪽같은그녀,1130,ms(misu****),진짜 믿고보는 나문희선생님 영화인데 너무나도 감동적이였다! 처음에는 기대안하고 보앗다가 너무나도 감동받아서 울고 추억에 다시한번 눈물을 흘리게 하는 영화였다,10,1,2,2019-12-07 21:07:00,0                                         1
                  감쪽같은그녀,1143,지니홀릭(ksj1****),역시 명품배우 나문희!서글픈 감동이 있는 영화..,8,1,2,2019-12-06 19:05:00,0                                                                                                    1
                  감쪽같은그녀,1180,이윤정(jiss****),알바아니고 일반인인데 어떻게 보면 뻔한 스토리일수도있지만 그와중에 재미도 있고 마냥 뻔하지는 않아요!배우분들 연기 진짜 잘하심ㅠ 나문희 할머님 항상 건강하시고 오래오래 만수무강하세요~마니 감동이고 슬프니까 휴지필수....,10,2,3,2019-12-07 13:59:00,0    1
                  감쪽같은그녀,1213,aocl****,"공주의 연기에 박수를! 남자 아역배우와의 케미도 좋았고 귀여워서 웃음이 절로 나오더라구요~또, 나문희씨의 표현력은 정말 최고에요 가슴이 미어 지도록 슬펐습니다ㅜㅜ감정이입도 잘되고, 할머니 생각나서 내일 당장 뵈러 가...",9,3,4,2019-12-06 01:31:00,0     

In [22]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [23]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소 품사          분류                                                     
!   Punctuation 감쪽같은그녀,1130,ms(misu****),진짜 믿고보는 나문희선생님 영화인데 너무...    1
                감쪽같은그녀,1143,지니홀릭(ksj1****),역시 명품배우 나문희!서글픈 감동이 ...    1
                감쪽같은그녀,1180,이윤정(jiss****),알바아니고 일반인인데 어떻게 보면 뻔한...    1
                감쪽같은그녀,1213,aocl****,"공주의 연기에 박수를! 남자 아역배우와의 케미...    1
                감쪽같은그녀,1223,남하진(haji****),"진짜 기대없이 스토리도 모르고 봤는데...    2
...                                                                 ...
힝힝  Noun        오문희,1191,힝힝(gzyq****),가족과 함께 보기 좋은 영화입니다,10,0,0...    1
힣힣  Exclamation 결백,1838,백산(sans****),신혜선 졸라 이쁨 히힣힣,10,2,0,2020-...    1
   Foreign     그대를사랑,",7,0,1,2011-03-31 20:54:00,0                  1
女   Foreign     내머리속에,4328,prom****,애인(女)의 우는 모습이 보고 싶다면... 안울면...    1
！   Foreign     그대를사랑,6787,sumi****,"어제 시사회 보고 왔어요, <명품연기, 깨알웃음...    1

[646732 rows x 1 columns]

In [24]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')